In [6]:
import numpy as np
import copy
def divide(C):
    #input: cost matrix
    n = len(C)
    used = []
    total = set([i for i in range(1,n+1)])
    i = 1
    while i <= n:
        all = []
        remain = list(total - set(used))
        for j in remain:
            used.append(j)
            sample = C[i-1,j-1]
            remain_1 = set(remain)-set([j])
            for k in range(i+1,n+1):
                r = remain_1.pop()
                sample += C[k-1,r-1]
            all.append((sample,j))
            used.pop()
        if i == 1: #无回溯。
            next = all.index(min(all))
            used.append(all[next][1])
            i += 1
        else:#回溯。
            remain = total - set(used)
            new = remain.pop()
            now = used.pop()
            remain.add(now)
            sample = C[i-2,new-1]-C[i-2,now-1]
            for k in range(i,n+1):
                r = remain.pop()
                sample += C[k-1,r-1]
            all.append((sample,n+1))
            next = all.index(min(all))
            used.append(now)
            if all[next][1] == n+1:
                used.pop()
                i -= 1
            else:
                used.append(all[next][1])
                i += 1
    return used

if __name__ == '__main__':
    C = np.random.rand(5,5)
    print(list(enumerate(divide(C), start=1)))

[(1, 4), (2, 2), (3, 1), (4, 5), (5, 3)]


In [1]:
import numpy as np
import copy
def Hungarian(C):
    #input: cost matrix
    def count_zeros_row(C):#计算每一行0的个数。
        n = len(C)
        zeros_row = [] 
        for i in range(n):
            count = 0
            for j in range(n):
                if C[i,j] == 0:
                    count += 1
            zeros_row.append(count)
        return zeros_row
    
    def count_zeros_col(C):#计算每一列0的个数。
        n = len(C)
        zeros_col = [] 
        for i in range(n):
            count = 0
            for j in range(n):
                if C[j,i] == 0:
                    count += 1
            zeros_col.append(count)
        return zeros_col
    
    def mark_zero(C, zeros_row, zeros_col, mark): #对C中的0进行标记。
        #以下先标记每行单独的未标记0为1，并把此0对应列上未标记的0标记为-1。再对每列做同样操作。
        n = len(C)
        for i in range(n):
            if zeros_row[i] == 1:
                for j in range(n):
                    if C[i,j] == 0 and (i,j) not in mark:
                        mark[(i,j)] = 1
                        zeros_row[i] -= 1
                        zeros_col[j] -= 1
                        for s in range(n):
                            if C[s,j] == 0 and (s,j) not in mark:
                                mark[(s,j)] = -1
                                zeros_row[s] -= 1
                                zeros_col[j] -= 1
                        break
        for j in range(n):
            if zeros_col[j] == 1:
                for i in range(n):
                    if C[i,j] == 0 and (i,j) not in mark:
                        mark[(i,j)] = 1
                        zeros_row[i] -= 1
                        zeros_col[j] -= 1
                        for s in range(n):
                            if C[i,s] == 0 and (i,s) not in mark:
                                mark[(i,s)] = -1
                                zeros_row[i] -= 1
                                zeros_col[s] -= 1
                        break
        #以下考虑行、列中有多于一个未标记0的情形。此时任选一个标记为1，并将其同行同列的未标记0标记为-1。
        for i in range(n):
            if zeros_row[i] >=2:
                for j in range(n):
                    if C[i,j] == 0 and (i,j) not in mark:
                        mark[(i,j)] = 1
                        zeros_row[i] -= 1
                        zeros_col[j] -= 1
                        for s in range(n):
                            if C[s,j] == 0 and (s,j) not in mark:
                                mark[(s,j)] = -1
                                zeros_row[s] -= 1
                                zeros_col[j] -= 1
                        for u in range(n):
                            if C[i,u] == 0 and (i,u) not in mark:
                                mark[(i,u)] = -1
                                zeros_row[i] -= 1
                                zeros_col[u] -= 1
                        break
        for j in range(n):
            if zeros_col[j] >=2:
                for i in range(n):
                    if C[i,j] == 0 and (i,j) not in mark:
                        mark[(i,j)] = 1
                        zeros_row[i] -= 1
                        zeros_col[j] -= 1
                        for u in range(n):
                            if C[i,u] == 0 and (i,u) not in mark:
                                mark[(i,u)] = -1
                                zeros_row[i] -= 1
                                zeros_col[u] -= 1
                        for s in range(n):
                            if C[s,j] == 0 and (s,j) not in mark:
                                mark[(s,j)] = -1
                                zeros_row[s] -= 1
                                zeros_col[j] -= 1
                        break
    
    def select(C, mark):
        n = len(C)
        finrow = set()
        fincol = set()
        for element in mark:
            if mark[element] == 1:
                finrow.add(element[0])        
        unmrow = set(range(n)) - finrow
        count = 1
        while count > 0:
            count = 0
            for i in unmrow:
                for j in range(n):
                    if C[i,j] == 0 and j not in fincol:
                        fincol.add(j)
                        count += 1
            for j in fincol:
                for i in range(n):
                    if C[i,j] == 0 and (i,j) in mark and mark[(i,j)] == 1 and i not in unmrow:
                        unmrow.add(i)
                        count += 1
        unmcol = set(range(n)) - fincol
        remain = [[C[i,j] for j in unmcol] for i in unmrow]
        le = len(remain)
        if le:
            mi = float('inf')
            for i in range(le):
                mi = min(mi, min(remain[i]))
            for i in unmrow:
                for j in range(n):
                    C[i,j] -= mi
            for j in fincol:
                for i in range(n):
                    C[i,j] += mi
        
        
    
    def subtract(C):
        n = len(C)
        for i in range(n):#对每一行减去最小值。
            mi = min(C[i,:])
            for j in range(n):
                C[i,j] -= mi
        for i in range(n):#对每一列减去最小值。
            mi = min(C[:,i])
            for j in range(n):
                C[j,i] -= mi

    n = len(C)
    
    subtract(C)
    while True:
        row = count_zeros_row(C)
        col = count_zeros_col(C)
        mark = dict() #记录0被标记的状态。
        mark_zero(C, row, col, mark)
        select(C, mark)
        result = []
        count = 0
        for i,j in mark:
            if mark[(i,j)] == 1:
                count += 1
                result.append((i+1,j+1))
        if count == n:
            return result
        else:
            continue
    
        
if __name__ == '__main__':
    C = np.random.rand(512,512)
    print(Hungarian(C))
    #which takes around 10 minutes.

[(1, 395), (2, 209), (4, 369), (6, 293), (7, 54), (10, 390), (11, 331), (12, 144), (13, 200), (14, 39), (15, 235), (20, 283), (21, 161), (23, 374), (24, 69), (25, 473), (26, 415), (30, 232), (32, 274), (33, 364), (34, 367), (39, 93), (40, 218), (41, 455), (42, 388), (43, 210), (47, 320), (50, 411), (51, 107), (52, 212), (55, 207), (56, 466), (59, 470), (60, 321), (61, 169), (62, 65), (65, 61), (69, 488), (71, 405), (72, 183), (73, 356), (77, 386), (78, 22), (80, 468), (81, 192), (83, 333), (86, 49), (88, 134), (90, 478), (92, 57), (95, 371), (97, 275), (98, 236), (99, 102), (100, 251), (101, 297), (106, 503), (109, 128), (111, 82), (112, 494), (113, 446), (114, 259), (119, 504), (120, 282), (121, 256), (122, 72), (123, 428), (124, 391), (125, 359), (128, 121), (129, 174), (134, 378), (135, 432), (136, 94), (138, 111), (140, 127), (141, 117), (142, 434), (143, 201), (144, 20), (145, 451), (146, 456), (148, 246), (150, 205), (152, 308), (155, 250), (156, 313), (157, 363), (158, 231), (16